In [9]:
from langchain.tools import StructuredTool, tool

The `@tool` decorator is the simplest way to define a custom Tool.

The function name will be used as the Tool name, but it can be overridden by passing a string as the first argument.
Additionally, the function's docstring will be used as the Tool's `description`, so a docstring **MUST** be included.

## 1. Creation with @tool wrapper

In [10]:
@tool
def xyz(query: str) -> str:
    """Descrizione in linguaggio naturale di che cosa fa questo tool"""
    return "Questo è l'output del tool"


print(f"Tool:         [{xyz.name}]\nDescrizione:  {xyz.description}\nInput schema: {xyz.args}")

Tool:         [xyz]
Descrizione:  Descrizione in linguaggio naturale di che cosa fa questo tool
Input schema: {'query': {'title': 'Query', 'type': 'string'}}


In [11]:
xyz.run( {"query":"ciao"} )

"Questo è l'output del tool"

## 2. Creation with StructuredTool

In [12]:
def xyz(query):
    return "Questo è l'output del nuovo tool"

xyz_tool = StructuredTool.from_function(
    func=xyz,
    name="xyz-tool",
    description="Descrizione in linguaggio naturale di che cosa fa questo tool",
    # coroutine= ... eventuale metodo per gestire chiamate asincrone, se disponibile. Questo possiamo farlo solo con questa modalità di creazione tool
)

print(f"Tool:         [{xyz_tool.name}]\nDescrizione:  {xyz_tool.description}\nInput schema: {xyz_tool.args}")

Tool:         [xyz-tool]
Descrizione:  Descrizione in linguaggio naturale di che cosa fa questo tool
Input schema: {'query': {'title': 'Query'}}


In [13]:
xyz_tool.run( {"query":"ciao"} )

"Questo è l'output del nuovo tool"

## Add a tool to an LLM

In [15]:
from langchain_experimental.tools import PythonREPLTool


In [29]:
print(PythonREPLTool().description)

A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.


In [35]:
@tool
def censor(query: str) -> str:
    """Questo tool censura le risposte del bot quando si parla di Sanremo """
    return "Preferisco non parlare di questi argomenti"



In [36]:
tools = []
tools.append(censor)
tools.append(PythonREPLTool())

In [37]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that use step by step process to handle the user input"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)


In [38]:
# Load variables from .env
load_dotenv()

# Retrieve variables
api_key = os.getenv("API_KEY")


In [39]:
# Creation of the agent with the tool
# https://api.python.langchain.com/en/latest/agents/langchain.agents.tool_calling_agent.base.create_tool_calling_agent.html

model = ChatOpenAI(model="gpt-4o", # GPT-3.5-turbo is the default model
                 openai_api_key=api_key,
                 temperature=.75,
                 max_tokens=1024,
                 request_timeout=30)  

agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)


Among the various actions performed by the agent_executor, it also takes care of filling the agent's "notebook" from time to time, through an input called `intermediate_steps` which will contain the "log" of the agent's "reasoning" history.

If we call the agent directly by hand (without going through the agent_executor), we must manually pass this information (which will obviously be empty as there is no history).

If the agent deems it necessary, it will perform a specific action through the use of a tool and will return an object of type __ToolAgentAction__ specifying the tool to be used and the input to be passed to the tool so that the agent_executor can handle it accordingly.

In [40]:
agent.invoke({"input": 'calcola un decimo del valore riportato nel seguente testo: "abbiamo speso 25€ in totale"', "intermediate_steps":""})

[ToolAgentAction(tool='Python_REPL', tool_input={'query': 'total_spent = 25\none_tenth = total_spent / 10\nprint(one_tenth)'}, log="\nInvoking: `Python_REPL` with `{'query': 'total_spent = 25\\none_tenth = total_spent / 10\\nprint(one_tenth)'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_78jIUL4IzUrX9feGVgBgtcgR', 'function': {'arguments': '{"query":"total_spent = 25\\none_tenth = total_spent / 10\\nprint(one_tenth)"}', 'name': 'Python_REPL'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 149, 'total_tokens': 188, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f9f4fb6dbf', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-fe2df0a3-2930-4153-b41a-9

In [41]:
agent_executor.invoke({"input": 'calcola un decimo del valore riportato nel seguente testo: "abbiamo speso 25€ in totale"'})



> Entering new AgentExecutor chain...

Invoking: `Python_REPL` with `{'query': 'spesa_totale = 25\nun_decimo = spesa_totale / 10\nun_decimo'}`




> Finished chain.


{'input': 'calcola un decimo del valore riportato nel seguente testo: "abbiamo speso 25€ in totale"',
 'output': ''}

In [42]:
agent.invoke({"input": 'Chi ha vinto Sanremo?', "intermediate_steps":""})

[ToolAgentAction(tool='censor', tool_input={'query': 'Chi ha vinto Sanremo?'}, log="\nInvoking: `censor` with `{'query': 'Chi ha vinto Sanremo?'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_W9OCs3IoEpJepfmNGRKfIK37', 'function': {'arguments': '{"query":"Chi ha vinto Sanremo?"}', 'name': 'censor'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 131, 'total_tokens': 152, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f9f4fb6dbf', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-db7bcfb1-f18a-47e6-83f5-41d6c96fd0fe-0', tool_calls=[{'name': 'censor', 'args': {'query': 'Chi ha vinto Sanremo?'}, 'id': 'call_W9OCs3IoEpJepfmNGRKfIK37', 'type': 'tool_call

In [43]:
agent_executor.invoke({"input": 'Chi ha vinto Sanremo?'})



> Entering new AgentExecutor chain...

Invoking: `censor` with `{'query': 'Chi ha vinto Sanremo?'}`


Preferisco non parlare di questi argomentiPreferisco non parlare di questi argomenti.

> Finished chain.


{'input': 'Chi ha vinto Sanremo?',
 'output': 'Preferisco non parlare di questi argomenti.'}